<a href="https://colab.research.google.com/github/FariusGitHub/DataScience/blob/master/SOUP2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import bs4
import requests
import re
from datetime import datetime, timedelta
from tqdm import tqdm
import pandas as pd

DSW1=[]; DST1=[]; DSS1= []; DSL1= []; DSC1=[]; DSD1=[]; DSE1=[]; 
soup = bs4.BeautifulSoup(requests.get('https://ca.indeed.com/jobs?q=data+scientist&l=Toronto,+ON&start=').text, 'lxml')

for l in tqdm(range(0,int(soup.find('div', id='searchCountPages').text\
                          .strip('\n Page jobs').split(' of')[1])//50*50,50)):
    start = l
    soup = bs4.BeautifulSoup(requests.get('https://ca.indeed.com/jobs?q=data+\
            scientist&l=Toronto,+ON&start='+str(start)+'&limit=50').text, 'lxml')

 
    jk=[]; fccid=[]; 
    for i in range(len(soup.select('.title'))):
        j = 0 if soup.select('.date')[i].text.strip()[:2].isalpha() \
            else int(soup.select('.date')[i].text.strip()[:2])
        k=(datetime.today().date()-timedelta((j))).strftime("%x")
        jk += [str(soup.find_all("script", text=re.compile("jobmap"), \
              type="text/javascript")[0]).split('\n\n')[i+3]\
              .split("]= {jk:'")[1].split(",cmpid:'")[0].split("',efccid:")[0]]
        fccid += [str(soup.find_all("script", text=re.compile("jobmap"), \
                type="text/javascript")[0]).split('\n\n')[i+3]\
                .split("]= {jk:'")[1].split(",cmpid:'")[1].split("',num:'")[0]]
        DSW1+=['http://ca.indeed.com/rc/clk?jk='+jk[i]+'&fccid='+fccid[i]+'&vjs=3'] #WEB
        DST1+=[soup.select('.title')[i].a.get('title')]                             #TITLE
        DSS1+=[soup.select('.summary')[i].text.strip('\n')]                         #SUMMARY
        DSL1+=[soup.select('.sjcl')[i](class_='recJobLoc')[0].get('data-rc-loc')]   #LOC
        DSC1+=[soup.select('.sjcl')[i](class_='company')[0].text.strip('\n')]       #COMPANY 
        DSD1+=['>=30days' if j == 30 else k]                                        #DATE 
        try:
            DSE1 += ['<p>'.join(str(bs4.BeautifulSoup(requests.get(DSW1[i])\
            .text, 'lxml').select(".jobsearch-jobDescriptionText")[0])\
            .split('</p>')).lstrip('<div class="jobsearch-jobDescriptionText" id=\
                  "jobDescriptionText"><p>')]
        except Exception:
            DSE1 += ['error during downloading']            #DESC


  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
for i in range(len(DSE1)):
    print(i, DSC1[i], DSW1[i], DSE1[i][:20])

0 Boast.AI http://ca.indeed.com/rc/clk?jk=556354c221e89af5&fccid=8c078e5ac86534ee&vjs=3 Each year the US and
1 Staffinity Inc. http://ca.indeed.com/rc/clk?jk=63a9885a9063bbb1&fccid=640f182ba57686f6&vjs=3 About Staffinity Inc
2 TELUS http://ca.indeed.com/rc/clk?jk=bbe976417dee58db&fccid=1f30f2e61ba7b1e1&vjs=3 Scientist<p>
<p>Toda
3 Arteria AI http://ca.indeed.com/rc/clk?jk=7b3955a34f82318a&fccid=18c1c5115b671740&vjs=3 About Arteria AI</b>
4 QuadReal http://ca.indeed.com/rc/clk?jk=eefa70bfa123e2a3&fccid=f049514aadb99749&vjs=3 About QuadReal</b><p
5 Gannett http://ca.indeed.com/rc/clk?jk=93043fc73e0cfd85&fccid=1230acb7e56c6df5&vjs=3 Gannett Co., Inc. (N
6 Veeva Systems http://ca.indeed.com/rc/clk?jk=acdc01645d55764a&fccid=353eb997fc901045&vjs=3 At Veeva, we build e
7 TELUS http://ca.indeed.com/rc/clk?jk=b56337f9a96cd34a&fccid=1f30f2e61ba7b1e1&vjs=3 Create awesome exper
8 QuantumSec Inc. http://ca.indeed.com/rc/clk?jk=d413cdd4ae579b1e&fccid=a10cc1e8f0bb2a1e&vjs=3 POSITION DESCRIPTION
9 fgf

In [ ]:
df = pd.DataFrame(list(zip(DSD1, DSC1, DST1, DSL1, DSS1, DSE1, DSW1)), 
      columns =['Posted', 'Company', 'Job Title', 'Location', 'Summary', 'Desc', 'url'])

len(df.drop_duplicates(subset='url'))

197

In [ ]:
len(df)

209